## ROME TO FAP LINKING TABLE PROCESSING

### Purpose
This script processes the ROME to FAP linking table, structuring occupational classification codes for analysis.

### Steps
1. **Load Data:** Import the Excel file and select the relevant sheet.
2. **Extract Categories:** Assign hierarchical FAP codes (`fap22`, `fap87`, `fap225`).
3. **Handle Missing Data:** Remove empty rows and forward-fill missing values.
4. **Assign Professional Families:** Map `famille_pro` names based on category levels.
5. **Clean & Filter:** Keep relevant rows, drop unnecessary columns, and reorder data.

### Output
A structured dataset with FAP codes, professional families, PCS categories, and ROME classifications for streamlined analysis and merging.


### 1- LIBRARIES AND PATHS

In [1]:
# IMPORT LIBRARIES
from pathlib import Path # To set relative paths
import pandas as pd

In [2]:
# GETTING PROJECT'S ROOT DIRECTORY
base_folder = Path().resolve()  # CURRENT WORKING DIRECTORY
main_folder = base_folder.parent

# SETTING PATHS AND DIRECTORIES
fap_to_rome = main_folder / "data" / "linking tables" / "Rome-V3 vers Fap-2009.xls"
output_path = main_folder / "data" / "linking tables" / "Rome_to_Fap_processed.csv"

### 2- WORKFLOW

In [3]:
# IMPORT ROME TO FAP LINKING TABLE
link = pd.read_excel(fap_to_rome, sheet_name=1)

# ASSIGNING CATEGORY (FAP COLUMN)
category_col = link.columns[0]

# FUNCTION TO EXTRACT CATEGORIES IN SEPARATE COLUMNS
def extract_categories(code):
    if pd.isna(code):
        return pd.Series([None, None, None])  # Handle missing values
    elif len(code) == 1:  # Level 1 (e.g., 'A')
        return pd.Series([code, None, None])
    elif len(code) == 3:  # Level 2 (e.g., 'A0Z')
        return pd.Series([code[0], code, None])
    else:  # Level 3 (e.g., 'A0Z00')
        return pd.Series([code[0], code[:3], code])

# APPLY FUNCTION TO CREATE NEW CATEGORY COLUMN
link[['fap22', 'fap87', 'fap225']] = link[category_col].apply(extract_categories)

# DROP BLANK ROWS
link.dropna(subset=['fap22', 'fap87', 'fap225', 'ROME'], how='all', inplace=True)

# CREATING A FLAG FOR MISSING ROME CODE (TO USE LATER FOR DROP)
link["no_rome_code"] = link["ROME"].isna()

# CREATE COLUMNS FOR PROFESSIONAL FAMILY FOR EACH FAP CATEGORY
link["famille_pro22"] = link["Familles professionnelles"].where(link["fap22"].notna() & link["fap87"].isna() & link["fap225"].isna())
link["famille_pro87"] = link["Familles professionnelles"].where(link["fap22"].notna() & link["fap87"].notna() & link["fap225"].isna())
link["famille_pro225"] = link["Familles professionnelles"].where(link["fap22"].notna() & link["fap87"].notna() & link["fap225"].notna())

# REPLACE NA WITH CLOSEST VALUE UPWARD
link = link.fillna(method="ffill")

# KEEP COLUMNS AND DATA OF INTEREST
link = link[link["no_rome_code"] == False]
link = link[["fap22", "famille_pro22", "fap87", "famille_pro87", "ROME", 
             "Répertoire Opérationnel des Métiers et des Emplois"]]
link.rename(columns={"ROME": "rome", "Répertoire Opérationnel des Métiers et des Emplois": "rome_label"}, inplace=True)

# DROP DUPLICATES
link.drop_duplicates(inplace=True)

# DROP ASTERISKS IN THE FAMILLE PRO COLUMNS
link['famille_pro22'] = link['famille_pro22'].str.replace('*', '', regex=False)
link['famille_pro87'] = link['famille_pro87'].str.replace('*', '', regex=False)

# PRINT THE TABLE
link

,fap22,famille_pro22,fap87,famille_pro87,rome,rome_label
25,A,"Agriculture, marine, pêche",A0Z,"Agriculteurs, éleveurs, sylviculteurs, bûcherons",A1416,"Polyculture, élevage"
27,A,"Agriculture, marine, pêche",A0Z,"Agriculteurs, éleveurs, sylviculteurs, bûcherons",A1403,Aide d'élevage agricole et aquacole
28,A,"Agriculture, marine, pêche",A0Z,"Agriculteurs, éleveurs, sylviculteurs, bûcherons",A1407,Élevage bovin ou équin
29,A,"Agriculture, marine, pêche",A0Z,"Agriculteurs, éleveurs, sylviculteurs, bûcherons",A1408,Élevage d'animaux sauvages ou de compagnie
30,A,"Agriculture, marine, pêche",A0Z,"Agriculteurs, éleveurs, sylviculteurs, bûcherons",A1409,Élevage de lapins et volailles
...,...,...,...,...,...,...
1401,W,"Enseignement, formation",W0Z,Enseignants,K2109,Enseignement technique et professionnel
1405,W,"Enseignement, formation",W0Z,Enseignants,K2103,Direction d'établissement et d'enseignement
1410,W,"Enseignement, formation",W0Z,Enseignants,K2108,Enseignement supérieur
1416,W,"Enseignement, formation",W1Z,Formateurs,K2110,Formation en conduite de véhicules


The issue is that the there exists different fap225 with the same **rome code**, the only think that differentiate them is their associated "qualification code" which is not available neither in our STMT or JOCAS dataset, where we only have the rome code.

In [4]:
# NUMBER OF DUPLICATED ROME (WITHOUT THE FIRST VALUE)
print(f"Duplicated ROME with different FAP : {link[link['rome'].duplicated()]['rome'].nunique()}")

Duplicated ROME with different FAP : 168


We have 168 duplicated rome code out of 714 ones. We will assume that people with similar ROME code (even across different FAP) have the same capabilities and could therefore apply to any job with this ROME code --> this will create some duplicates for the same ROME code.

### 3- EXPORT FILE

In [5]:
# EXPORT DATASET 
link.to_csv(output_path, index=False)